In [118]:
# Instalo el modulo
import pandas as pd
from pandas.io.json import json_normalize
import pymongo
from pymongo import MongoClient
from pprint import pprint
from geojson import Point
import numpy as np


In [119]:
client = MongoClient('localhost',27017)
db = client.local
collection = db.mybbd

In [120]:
print(collection.count_documents({}))

18801


 1. From all the companies get first office, number of people and company name

In [121]:
q=collection.find(projection={'name':1,'offices':1,'number_of_employees':1,'_id':0})
df=pd.DataFrame(list(q))
df.head()


,name,number_of_employees,offices
0,Wetpaint,47.0,"[{'description': '', 'address1': '710 - 2nd Av..."
1,Digg,60.0,"[{'description': None, 'address1': '135 Missis..."
2,Zoho,1600.0,"[{'description': 'Headquarters', 'address1': '..."
3,Omnidrive,NaN,"[{'description': '', 'address1': 'Suite 200', ..."
4,Postini,NaN,"[{'description': None, 'address1': '959 Skyway..."


In [122]:
result=(df.loc[65,'offices'][0]['latitude'],df.loc[65,'offices'][0]['latitude'])
result

(47.620716, 47.620716)

In [123]:
def first_office_point(x): 
    # {'geometry': {'type': 'Point', 'coordinates': [-70.5867, -29.9953, 760]}, 'type': 'Feature', 'properties': {'name': 'A', 'description': 'Place a'}}
    try: 
        point = Point([x[0]['longitude'], x[0]['latitude']]) 
    except: 
        point = np.nan
    else: 
        pass
    finally: 
        return point

In [124]:
# aplicamos el constructor de Point a la nueva columna
df['geopoint'] = df['offices'].apply(lambda x: first_office_point(x))

In [125]:
# eliminamos officessss y los registros que tienen NaN

# df.drop(['B', 'C'], axis=1)

df = df.drop(['offices'], axis=1) 
df_clean = df.dropna()
df_clean.head()

,name,number_of_employees,geopoint
0,Wetpaint,47.0,"{'type': 'Point', 'coordinates': [-122.333253,..."
1,Digg,60.0,"{'type': 'Point', 'coordinates': [-122.394523,..."
2,Zoho,1600.0,"{'type': 'Point', 'coordinates': [-121.904945,..."
5,Geni,18.0,"{'type': 'Point', 'coordinates': [-118.393064,..."
7,AdventNet,600.0,"{'type': 'Point', 'coordinates': [-121.904945,..."


In [130]:
# jotason = df_clean.to_json(orient='records')

# jotason
#ahora guardamos el archiov y lo metemos en la BBDD
#df_clean.to_json('geo.json', orient='records')

In [133]:
with open("demofile2.json", "w") as f:
    for i,e in df_clean.iterrows():
        #print(e.to_json())
        f.write(e.to_json()+"\n")